In [1]:
import numpy as np
import pandas as pd
import os
import csv
import time
import glob
from datetime import datetime


In [2]:
def caminho():
    import socket
    name = socket.gethostname()
    if name == 'LGNTVIVO71':
        path = "\\\\desktop-73e3j72\\teste_internet\\"
        #path = "C:\\Users\\fflose\\Lab\\\Pessoal\\python\\csv\\teste_internet\\"
    else:
        path = "\\\\desktop-73e3j72\\teste_internet\\"
    return path

In [3]:
def lista_arquivos():
        path = caminho()
        ListaArquivos = os.listdir(path)
        l = []
        ListaArquivos = os.listdir(path)
        l = ListaArquivos
        dt = datetime.now().strftime('%d-%m-%Y')
        for x in l:
    #print(x[15:25])
                if x[15:25] == dt:
                        l.append(x)
                 
        return l


In [4]:
def deleta_vazio():
    for x in lista_arquivos():
        path = caminho()
        arq1 = open(path+x)
        arq1Lendo = csv.reader(arq1,delimiter=',',quoting=csv.QUOTE_NONE)
        arqLista = list(arq1Lendo)
        if len(arqLista) == 1:
            arq1.close()
            os.remove(path+x)        
            return print('arquivo '+ x +'  deletado!') 

In [5]:
def lendo_arquivo():
    #l = []
    path = caminho()
    os.chdir(path)
    extension = 'csv'
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    dt = datetime.now().strftime('%d-%m-%Y')
    l = []
    for x in all_filenames:
        #print(x[15:25])
        if x[15:25] == dt:
                    l.append(x)
    #combinar todos os arquivos da lista
    combined_csv = pd.concat([pd.read_csv(f) for f in l ])
    #exportar para csv
    #combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')
    df = pd.DataFrame(combined_csv)
    df.insert(4,'min_fechado',df['hora_teste'].str[3:5])
    #df.insert(2,'min_fechado',df['hora_teste'].str[3:5])
    return df

In [6]:
def resultado():
    deleta_vazio()
    df = []
    df = pd.DataFrame(lendo_arquivo())
    
#qtd de teste de download por hora
    dfqtdTeste =  df.groupby(['dt_teste','hora_fechada','min_fechado']) ['down'].count().reset_index()
    dfqtdTeste.rename(columns={'dt_teste':'data', 'hora_fechada':'hora','min_fechado':'minuto','down':'qtdteste'}, inplace = True)
    dfqtdTeste = pd.pivot_table(dfqtdTeste, values = 'qtdteste', index=['data','hora','minuto']).reset_index()  
#max de download por hora
    dfqmedia =  df.groupby(['dt_teste','hora_fechada','min_fechado']) ['down'].mean().reset_index()
    dfqmedia.rename(columns={'dt_teste':'data', 'hora_fechada':'hora','min_fechado':'minuto' ,'down':'media'}, inplace = True)
    dfqmedia = pd.pivot_table(dfqmedia, values = 'media', index=['data','hora','minuto']).reset_index()
#min de download por hora
    dfqtdmin =  df.groupby(['dt_teste','hora_fechada','min_fechado']) ['down'].min().reset_index()
    dfqtdmin.rename(columns={'dt_teste':'data', 'hora_fechada':'hora' ,'min_fechado':'minuto' ,'down':'min'}, inplace = True)
    dfqtdmin = pd.pivot_table(dfqtdmin, values = 'min', index=['data','hora','minuto']).reset_index()
#max de download por hora
    dfqtdmax =  df.groupby(['dt_teste','hora_fechada','min_fechado']) ['down'].max().reset_index()
    dfqtdmax.rename(columns={'dt_teste':'data', 'hora_fechada':'hora' ,'min_fechado':'minuto' ,'down':'max'}, inplace = True)
    dfqtdmax = pd.pivot_table(dfqtdmax, values = 'max', index=['data','hora','minuto']).reset_index()
#qtd chegou de download por hora
    dfqtdchegou =  df.groupby(['dt_teste','hora_fechada','min_fechado']) ['chegou'].sum().reset_index()
    dfqtdchegou.rename(columns={'dt_teste':'data', 'hora_fechada':'hora' ,'min_fechado':'minuto' ,'chegou':'qtdchegou'}, inplace = True)
    dfqtdchegou = pd.pivot_table(dfqtdchegou, values = 'qtdchegou', index=['data','hora','minuto']).reset_index()
    
#merge de todos
    dffinal1 = pd.merge(dfqtdTeste, dfqmedia, how = 'inner', on = ['hora','data','minuto'] ).reset_index()
    dffinal2 = pd.merge(dfqtdmin, dfqtdmax, how = 'inner', on = ['hora','data','minuto'] ).reset_index()
    dffinal3 = pd.merge(dffinal1, dffinal2, how = 'inner', on = ['hora','data','minuto'] ).reset_index()
    dffinal4 = pd.merge(dfqtdchegou, dffinal3, how = 'inner', on = ['hora','data','minuto'] )
    
    return(dffinal4[['data','hora', 'minuto', 'qtdteste','media', 'min','max','qtdchegou']])



In [7]:
def clear_console():
    os.system('clear')

In [8]:
df = resultado()
clear_console()
max_data = df['data'].max()
max_hora = df['hora'].max()
max_min = df['minuto'].max()
df = df.loc[df['data'] == max_data]
df = df.loc[df['hora'] == max_hora]
df = df.loc[df['minuto'] == max_min]
print(df)

arquivo teste_internet-02-08-2022-17-29-31.csv  deletado!


ValueError: No objects to concatenate

: 